In [2]:
import json
import pandas as pd

# Bentso technologies

In [88]:
bentso_label_uri=pd.read_csv('../final_tables/tables/bentso_activity_types_uri.csv')
bentso_label_uri.head(1)

,Bentso technology,uri
0,Fossil Hard coal,http://rdf.bonsai.uno/activitytype/entsoe/Foss...


In [89]:
exiobase2bentso=pd.read_csv('../final_tables/tables/exiobase_to_bentso_activities.csv')
exiobase2bentso.head(1)

,Exiobase Activity name,Bentso technology
0,Cultivation of paddy rice,NaN


In [90]:
exiobase_labels=pd.read_csv('../raw/exiobase_v3_3_17_activities.csv')
exiobase_labels.head(1)

,Activity name,Activity code 1,Activity code 2
0,Cultivation of paddy rice,i01.a,A_PARI


In [91]:
df=pd.merge(exiobase2bentso.dropna(),
         exiobase_labels,
         left_on='Exiobase Activity name',
         right_on='Activity name')\
.set_index('Exiobase Activity name')[['Activity code 2','Bentso technology']]

In [106]:
ex2bentso_act=pd.merge(df,bentso_label_uri,on='Bentso technology')
ex2bentso_act.head(1)

,Activity code 2,Bentso technology,uri
0,A_POWC,Fossil Hard coal,http://rdf.bonsai.uno/activitytype/entsoe/Foss...


In [108]:

ex2bentso_act=ex2bentso_act.rename({'Activity code 2':'brdfat',
                                    'Bentso technology':'rdfs:label',
                                    'uri':'brdfatb'
                                   },
                                   axis=1)
#ex2bentso_act.head(1)

ex2bentso_act.brdfatb=ex2bentso_act.brdfatb.str.extract('http://rdf.bonsai.uno/activitytype/entsoe/(.*)')

ex2bentso_act.head(1)

,brdfat,rdfs:label,brdfatb
0,A_POWC,Fossil Hard coal,Fossil_Hard_coal


In [123]:
ex2bentso_act

,brdfat,rdfs:label,brdfatb
0,A_POWC,Fossil Hard coal,Fossil_Hard_coal
1,A_POWC,Fossil Brown coal/Lignite,Fossil_Brown_coalorLignite
2,A_POWG,Fossil Gas,Fossil_Gas
3,A_POWG,Fossil Coal-derived gas,Fossil_Coal-derived_gas
4,A_POWN,Nuclear,Nuclear
5,A_POWH,Hydro Pumped Storage,Hydro_Pumped_Storage
6,A_POWH,Hydro Run-of-river and poundage,Hydro_Run-of-river_and_poundage
7,A_POWH,Hydro Water Reservoir,Hydro_Water_Reservoir
8,A_POWW,Wind Offshore,Wind_Offshore
9,A_POWW,Wind Onshore,Wind_Onshore


## turtle

In [145]:
f= open("../final_tables/tables_rdf_turtle/entsoe.ttl","w+")

f.write('@prefix bont: <http://ontology.bonsai.uno/core#> .\n')
f.write('@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n')
f.write('@prefix brdfatb: <http://rdf.bonsai.uno/activitytype/entsoe/> .\n')
f.write('@prefix brdfat: <http://rdf.bonsai.uno/activitytype/exiobase3_3_17/> .\n')
f.write('@prefix owl: <http://www.w3.org/2002/07/owl#> .\n')
f.write('@prefix dtype: <http://purl.org/dc/dcmitype/> .\n')
f.write('@prefix dc: <http://purl.org/dc/terms/> .\n')
f.write('@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n')
f.write('@prefix ns0: <http://purl.org/vocab/vann/> .\n')
f.write('@prefix cc: <http://creativecommons.org/ns#> .\n')
f.write('@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n')


f.write('\n')

f.write('<http://rdf.bonsai.uno/activitytype/entsoe/>\n')
f.write('  a dtype:Dataset ;\n')
f.write('  dc:title "Entsoe activity types"@en ;\n')
f.write('  dc:description "relation between entsoe electricity generators and exiobase"@en ;\n')
f.write('  foaf:homepage <http://rdf.bonsai.uno/activitytype/entsoe/documentation.html> ;\n')
f.write('  ns0:preferredNamespaceUri "http://rdf.bonsai.uno/activitytype/entsoe/#" ;\n')
f.write('  owl:versionInfo "Version 0.1 - 2019-03-25"@en ;\n')
f.write('  dc:modified "2019-03-25"^^xsd:date ;\n')
f.write('  dc:publisher "bonsai.uno" ;\n')
f.write('  dc:creator <http://bonsai.uno/foaf/bonsai.rdf#bonsai> ;\n')
f.write('  dc:contributor "Miguel F. Astudillo";\n')
f.write('  cc:license <http://creativecommons.org/licenses/by/3.0/> ;\n')
f.write('  rdfs:comment """First ever version 0.1 :Will change!"""@en .\n')

f.write('\n')

for r in ex2bentso_act.iterrows():  
    
    f.write('brdfatb:{} a bont:ActivityType;\n'.format(r[1]['brdfatb']))
    f.write('  owl:sameAs brdfat:{};\n'.format(r[1]['brdfat']))
    f.write('  rdfs:label:"{}".\n'.format(r[1]['rdfs:label']))
    f.write('\n')
    
f.close()

## JSON

```
"@graph": [
        {
      "@id": "brdfat:A_ALUM",
      "@type": "bont:ActivityType",
      "rdfs:label": "Aluminium production"
        },
```

now we need to say for each row that 

- brdfatb:Fossil_Brown_coalorLignite a bont:ActivityType;
- owl:sameAs brdfat:A_POWC;
- rdfs:label: "Fossil Hard coal".

In [35]:
ex2bentso_act.loc[0:1].to_dict()

{'Activity code 2': {0: 'A_POWC', 1: 'A_POWC'},
 'Bentso technology': {0: 'Fossil Hard coal', 1: 'Fossil Brown coal/Lignite'},
 'uri': {0: 'http://rdf.bonsai.uno/activitytype/bentso/Fossil_Hard_coal',
  1: 'http://rdf.bonsai.uno/activitytype/bentso/Fossil_Brown_coalorLignite'}}

In [42]:
json.dumps(ex2bentso_act.loc[0:1].to_dict())

'{"Activity code 2": {"0": "A_POWC", "1": "A_POWC"}, "Bentso technology": {"0": "Fossil Hard coal", "1": "Fossil Brown coal/Lignite"}, "uri": {"0": "http://rdf.bonsai.uno/activitytype/bentso/Fossil_Hard_coal", "1": "http://rdf.bonsai.uno/activitytype/bentso/Fossil_Brown_coalorLignite"}}'

In [38]:
json.dumps?

In [40]:
with open('example.json', 'w') as f: json.dump(ex2bentso_act.loc[0:1].to_dict(), f)

In [1]:
import json
from bontofrom.load_metadata import get_metadata
from pathlib import Path


output_dir = Path(__file__).parent / "output"


EXIOBASE_DOCKER = """
Please run the following to convert JSON-LD to TTL:
    cd "{0}"
    docker run -it --rm -v `pwd`:/rdf stain/jena riot -out Turtle bontofrom/output/flowobject.jsonld > bontofrom/output/flowobject.ttl
    docker run -it --rm -v `pwd`:/rdf stain/jena riot -out Turtle bontofrom/output/activitytype.jsonld > bontofrom/output/activitytype.ttl
    docker run -it --rm -v `pwd`:/rdf stain/jena riot -out Turtle bontofrom/output/location.jsonld > bontofrom/output/location.ttl
""".format((Path(__file__).parent.parent).absolute())


class Converter:
    def __init__(self, abbrev, full, filename, type_, metadata):
        self.context = {
                "bont" : "http://ontology.bonsai.uno/core#",
                "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
                "gn": "http://sws.geonames.org/",
                "schema": "http://schema.org/",
        }
        self.context.update({abbrev: full})
        self.metadata = metadata
        self.filename = filename
        self.type_ = type_

    def substitute(self, string):
        for k, v in self.context.items():
            string = string.replace(v, k + ":")
        return string

    def get_data(self):
        data = {
            "@context": self.context,
            "@graph": []
        }
        for name, uri in self.metadata[self.filename].items():
            data['@graph'].append({
                '@id': self.substitute(uri),
                "@type" : self.type_,
                "rdfs:label": name,
            })
        return data

    def write_file(self):
        with open(output_dir / (self.filename + ".jsonld"), "w", encoding='utf-8') as f:
            json.dump(self.get_data(), f,
                      ensure_ascii=False, indent=2)



def convert_exiobase():
    metadata = get_metadata()

    flow_object = Converter(
        "brdffo",
        "http://rdf.bonsai.uno/flowobject/exiobase3_3_17/",
        "flowobject",
        "bont:FlowObject",
        metadata,
    )
    flow_object.write_file()

    activity_type = Converter(
        "brdfat",
        "http://rdf.bonsai.uno/activitytype/exiobase3_3_17/",
        "activitytype",
        "bont:ActivityType",
        metadata,
    )
    activity_type.write_file()

    location = Converter(
        "brdfl",
        "http://rdf.bonsai.uno/location/exiobase3_3_17/",
        "location",
        "schema:Place",
        metadata,
    )
    location.write_file()

    print(EXIOBASE_DOCKER)
pass

ModuleNotFoundError: No module named 'bontofrom'